<a href="https://colab.research.google.com/github/NatashaB-randao/Formacao-Machine-Learning-Specialist/blob/main/ConfusionMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow